# GenAI and LlamaIndex ◦ March 6 2024 ◦ Mistplay

In [1]:
##################################################################
# Venue: Mistplay Offsite
# Talk: GenAI and LlamaIndex
# Speaker: Andrei Fajardo
##################################################################

![Title Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/title.excalidraw.svg)

![Title Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/framework.excalidraw.svg)

#### Notebook Setup & Dependency Installation

In [1]:
%pip install llama-index-vector-stores-qdrant llama-parse -q


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import nest_asyncio

nest_asyncio.apply()

## Motivation

![Motivation Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/motivation.excalidraw.svg)


In [39]:
# query an LLM and ask it about Mistplay
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4")
response = llm.complete("What does Mistplay do?")

In [40]:
print(response)

Mistplay is a loyalty platform for mobile gamers. Users earn points for playing new games and can exchange them for gift cards or in-game purchases. It also allows users to discover and play new games, participate in contests, and connect with other gamers.


In [41]:
response = llm.complete("What is the Mistplay Loyalty Index?")

In [42]:
print(response)

The Mistplay Loyalty Index is a system used by the mobile gaming platform, Mistplay, to measure a player's engagement and loyalty to a particular game. The index is calculated based on various factors such as the amount of time spent playing the game, the frequency of play, and the player's in-game progress. The higher the index, the more rewards the player can earn on the Mistplay platform.


In [43]:
response = llm.complete("What are the key survey insights from Mistplay's 2023 Mobile Gaming Loyalty Report?")

In [44]:
print(response)

As an AI, I can't provide real-time data or future predictions. As of my last update, it's still 2021, and I don't have access to Mistplay's 2023 Mobile Gaming Loyalty Report. I recommend checking Mistplay's official website or contacting them directly for the most accurate and up-to-date information.


## Basic RAG in 3 Steps

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/subheading.excalidraw.svg)


1. Build external knowledge (i.e., uploading updated data sources)
2. Retrieve
3. Augment and Generate

## 1. Build External Knowledge

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/step1.excalidraw.svg)

In [45]:
"""Load the data.

With llama-index, before any transformations are applied,
data is loaded in the `Document` abstraction, which is
a container that holds the text of the document.
"""

from llama_index.core import SimpleDirectoryReader

loader = SimpleDirectoryReader(input_dir="./data")
documents = loader.load_data()

In [46]:
# if you want to see what the text looks like
documents[0].text[:1000]

"2023\u2028\nMobile Gaming \u2028\nLoyalty Report\u2028\nDiscover what drives engagement and fosters \nstrong loyalty among today's mobile gamers\n"

In [47]:
"""Chunk, Encode, and Store into a Vector Store.

To streamline the process, we can make use of the IngestionPipeline
class that will apply your specified transformations to the
Document's.
"""

from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

client = qdrant_client.QdrantClient(location=":memory:")
vector_store = QdrantVectorStore(client=client, collection_name="test_store")

pipeline = IngestionPipeline(
    transformations = [
        SentenceSplitter(),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store,
)
_nodes = pipeline.run(documents=documents, num_workers=4)

In [48]:
# if you want to see the nodes
# len(_nodes)
_nodes[0].text

"2023\u2028\nMobile Gaming \u2028\nLoyalty Report\u2028\nDiscover what drives engagement and fosters \nstrong loyalty among today's mobile gamers"

In [49]:
"""Create a llama-index... wait for it... Index.

After uploading your encoded documents into your vector
store of choice, you can connect to it with a VectorStoreIndex
which then gives you access to all of the llama-index functionality.
"""

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

## 2. Retrieve Against A Query

![Step2 Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/step2.excalidraw.svg)

In [50]:
"""Retrieve relevant documents against a query.

With our Index ready, we can now query it to
retrieve the most relevant document chunks.
"""

retriever = index.as_retriever(similarity_top_k=2)
retrieved_nodes = retriever.retrieve("What are the key survey insights from Mistplay's 2023 Mobile Gaming Loyalty Report?")

In [51]:
# to view the retrieved node
print(retrieved_nodes[0].text[:500])

3 % ) .  A s  p l a y e r s  p r o g r e s s ,  t h e y  
a l s o  e n j o y  s e e i n g  t h e  n e w e s t  f e a t u r e s  a n d  u p d a t e s  a d d e d  t o  t h e  g a m e ,  w i t h  3 7 . 7 %  o f  p l a y e r s  c i t i n g  i t  a s  a  r e a s o n  f o r  r e t u r n i n g .  B u i l d i n g  c o r e  g a m e p l a y  l o o p s  t h a t  p r o v i d e  m e a n i n g f u l  a n d  v i s i b l e  p r o g r e s s i o n  –  c o m p l e t i n g  l e v e l s ,  u n l o c k i n g  n e w  


## 3. Generate Final Response

![Step3 Image](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/step3.excalidraw.svg)

In [52]:
"""Context-Augemented Generation.

With our Index ready, we can create a QueryEngine
that handles the retrieval and context augmentation
in order to get the final response.
"""

query_engine = index.as_query_engine()

In [16]:
# to inspect the default prompt being used
print(query_engine.
      get_prompts()["response_synthesizer:text_qa_template"]
      .default_template.template
)

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


In [53]:
response = query_engine.query("What are the key survey insights from Mistplay's 2023 Mobile Gaming Loyalty Report?")
print(response)

The key survey insights from Mistplay's 2023 Mobile Gaming Loyalty Report include motivations for continuing to play favorite games, such as the desire to keep progressing and reach new milestones, not wanting to abandon progress and achievements, enjoying new features and content updates, enjoying the competitiveness of the game and mastering skill levels, spending money in the games, believing the game is the best in its genre, playing because friends and/or family also play, and participating in the game's community. Additionally, the report highlights that mobile game advertising is the most successful method for players to discover new games, with nearly half relying on ads as a primary method, followed by app store exploration and direct recommendations from friends or family.


## Beyond Basic RAG: Improved PDF Parsing with LlamaParse

### The default pdf reader (PyPDF), like many out-of-the box pdf parsers struggle on complex PDF docs.

In [28]:
response = query_engine.query("What are the three listed key index insights from Mistplay's 2023 Mobile Gaming Loyalty Report?")
print(response)

The three listed key index insights from Mistplay's 2023 Mobile Gaming Loyalty Report are Loyalty Index, Loyalty Ratings, and Monetization Metrics.


In [36]:
response = query_engine.query("What are the top 3 categories of games which get the most loyalty (and list their index values) according to Mistplay's 2023 Mobile Gaming Loyalty Report?")
print(response)

RPG has the highest loyalty index value, followed by Strategy and Simulation in the top 3 categories of games according to Mistplay's 2023 Mobile Gaming Loyalty Report.


### Improved PDF Parsing using LlamaParse

In [23]:
from llama_parse import LlamaParse

parser = LlamaParse(result_type="markdown")
documents = parser.load_data(file_path="./data/2023 Mobile Gaming Loyalty Report by Mistplay-3.pdf")

Started parsing the file under job_id e256d886-1db2-449a-aaca-5d64200286ba
.....................

In [24]:
llama_parse_index = VectorStoreIndex.from_documents(documents)
llama_parse_query_engine = llama_parse_index.as_query_engine()

In [54]:
response = llama_parse_query_engine.query("What are the key survey insights from Mistplay's 2023 Mobile Gaming Loyalty Report?")
print(response)

Players are motivated by play-and-earn. Word of mouth is a top-3 acquisition source. Players play multiple games simultaneously but only spend money on a few. Additionally, players are interested in games where they can play and earn tangible rewards, with a significant percentage willing to switch from their current favorite game for this feature.


In [29]:
response = llama_parse_query_engine.query("What are the three listed key index insights from Mistplay's 2023 Mobile Gaming Loyalty Report?")
print(response)

RPG games garner the most loyalty. Strategy games drive average sessions per user. Lifestyle games see the highest repeat purchases.


In [35]:
response = llama_parse_query_engine.query("What are the top 3 categories of games which get the most loyalty (and list their index values) according to Mistplay's 2023 Mobile Gaming Loyalty Report?")
print(response)

RPG games have the highest loyalty rating of 75, followed by Strategy games with a loyalty rating of 70, and Simulation games with a loyalty rating of 67.


## In Summary

- LLMs as powerful as they are, don't perform too well with knowledge-intensive tasks (domain specific, updated data, long-tail)
- Context augmentation has been shown (in a few studies) to outperform LLMs without augmentation
- In this notebook, we showed one such example that follows that pattern.

# LlamaIndex Has More To Offer

- Data infrastructure that enables production-grade, advanced RAG systems
- Agentic solutions
- Newly released: `llama-index-networks`
- Enterprise offerings (alpha):
    - LlamaParse (proprietary complex PDF parser) and
    - LlamaCloud

### Useful links

[website](https://www.llamaindex.ai/) ◦ [llamahub](https://llamahub.ai) ◦ [llamaparse](https://github.com/run-llama/llama_parse) ◦ [github](https://github.com/run-llama/llama_index) ◦ [medium](https://medium.com/@llama_index) ◦ [rag-bootcamp-poster](https://d3ddy8balm3goa.cloudfront.net/rag-bootcamp-vector/final_poster.excalidraw.svg)